In [9]:
import pandas as pd
from collections import Counter
from string import digits
from keras.preprocessing import sequence


from pathlib import Path
folder_path=Path("Flickr8k/captions/")

In [3]:
def build_vocab(sentences,threshold):
    new_descrip=[]
    counter=Counter()
    for line in sentences:
        #tokenize sentence into words
        token=line.split()
        #lowercase words in list
        low_word=[word.lower() for word in token]
        #remove digits from the list
        remove_digits = str.maketrans('', '', digits)
        word_list= [i.translate(remove_digits) for i in low_word]
        #remove empty string elements from the list
        remove_space=list(filter(None,word_list))
        #update values in counter
        counter.update(remove_space)
        #make new list of cleaned descriptions
        new_descrip.append(" ".join(remove_space))
    #remove words from vocabulary if frequency is less than given threshold    
    vocabulary=[key for key,value in counter.items() if value>=threshold]    
        
    return vocabulary,new_descrip
        

In [4]:
pd_dataset=pd.read_csv("Flickr8k/captions/train_dataset.txt",delimiter="\t")
pd_frame=pd_dataset.values
print(pd_frame.shape)

(30000, 2)


In [5]:
sentences=[]
for sent in range(pd_frame.shape[0]):
    sentence=pd_frame[sent][1]
    sentences.append(sentence)#append the sentence in list "sentences"
len(sentences)   

30000

In [6]:
vocabulary,clean_desc=build_vocab(sentences,1)
len(vocabulary)

7668

In [7]:
#vectorization
word2id={value:index for index,value in enumerate(vocabulary)}
id2word={index:value for index,value in enumerate(vocabulary)}

In [8]:
word2id[id2word.get(0)]=7668
word2id["PAD"]=0

id2word[7668]=id2word.get(0)
id2word[0]="PAD"

In [9]:
vocab_size=len(id2word.keys())
vocab_size

7669

In [10]:
#find description with maximum length
max_len=0
for i in clean_desc:
    sent=i.split()
    if len(sent)>max_len:
        max_len=len(sent)
print(max_len)        

39


In [11]:
padded_seq,subseq_seq=[],[]
for line in range(len(clean_desc)):
    partial_seq=[]
    next_words=[]
    text=clean_desc[line].split()
    encod_text=[word2id[i] for i in text]#encode string to vectors
    for i in range(1,len(encod_text)):
        partial_seq.append(encod_text[:i])#list of lists of partial sentences in vector form
        next_words.append(encod_text[i])#store next word or target word in the list
    #post(to the right) padding sequences with zeros to fixed(maximum) length    
    partial_pad_seq=sequence.pad_sequences(partial_seq,max_len,padding="post")
    nextwords_hotvec = np.zeros([len(next_words), vocab_size], dtype=np.bool)#numpy array of falses 
    #vectorization
    for i,next_word in enumerate(next_words):
        nextwords_hotvec[i,next_word]=1
    padded_seq.append(partial_pad_seq)
    subseq_seq.append(nextwords_hotvec)
padded_seq=np.asarray(padded_seq)
subseq_seq=np.asarray(subseq_seq)

AttributeError: 'list' object has no attribute 'shape'

In [7]:
captions = np.zeros([0, max_len])#squeeze rows to zero(rows are not fixed)
next_words = np.zeros([0, vocab_size])
num_of_images=6000
for ix in range(num_of_images):
    #concatenate captions and next words of all the images
    captions = np.concatenate([captions, padded_sequences[ix]])
    next_words = np.concatenate([next_words, subsequent_words[ix]])

    
np.save(folder_path/captions.npy, captions)#save numpy array in npy file
np.save(folder_path/next_words.npy, next_words)


array([], shape=(0, 5), dtype=float64)